In [ ]:
from haystack import Pipeline

from talk_rag_scheduler.rag_pipeline import (
    create_rag_pipeline,
    prettify_rag_info,
    KEYWORD_RAG_PIPELINE_IMG,
    SEMANTIC_RAG_PIPELINE_IMG,
)
from talk_rag_scheduler.const import (
    OLLAMA_MODEL,
    OLLAMA_EMBEDDING_MODEL,
    NUM_PREDICT,
    TOP_K,
)
from talk_rag_scheduler.indexing_pipeline import (
    ingest_knowledge_and_run_indexing,
)
from talk_rag_scheduler.models import MetaType, RagOutput
from haystack.document_stores.in_memory import InMemoryDocumentStore

from IPython.display import Markdown, Image

## RAG with Keyword search

In [ ]:
semantic_document_store = InMemoryDocumentStore()
semantic_indexing_pipeline = ingest_knowledge_and_run_indexing(semantic_document_store)

In [ ]:
keyword_rag_pipeline: Pipeline = create_rag_pipeline(
    document_store=semantic_document_store, ollama_model=OLLAMA_MODEL
)
query = "How can I schedule an asynchronous job using the scheduler python library?"
results: RagOutput = keyword_rag_pipeline.run(
    {
        "retriever": {"query": query, "top_k": TOP_K},
        "prompt_builder": {"query": query},
        "llm": {"generation_kwargs": {"num_predict": NUM_PREDICT}},
    }
)
reply: str = results["llm"]["replies"][0]
meta: MetaType = results["llm"]["meta"][0]
Image(KEYWORD_RAG_PIPELINE_IMG)

In [ ]:
Markdown(reply)

In [ ]:
print(prettify_rag_info(meta))

## RAG with Semantic search

In [ ]:
semantic_document_store = InMemoryDocumentStore()
semantic_indexing_pipeline = ingest_knowledge_and_run_indexing(
    semantic_document_store,
    indexing_mode="semantic_split",
    ollama_embedding_model="mxbai-embed-large",
    split_by="passage",
    split_length=16,
    split_overlap=4,
    split_threshold=12,
)

In [ ]:
semantic_rag_pipeline: Pipeline = create_rag_pipeline(
    document_store=semantic_document_store,
    ollama_model=OLLAMA_MODEL,
    ollama_embedding_model=OLLAMA_EMBEDDING_MODEL,
    indexing_mode="semantic_split",
)
query = "How can I schedule an asynchronous job using the scheduler python library?"
semantic_results: RagOutput = keyword_rag_pipeline.run(
    {
        "retriever": {"query": query, "top_k": TOP_K},
        "prompt_builder": {"query": query},
        "llm": {"generation_kwargs": {"num_predict": NUM_PREDICT}},
    }
)
semantic_reply: str = semantic_results["llm"]["replies"][0]
semantic_meta: MetaType = semantic_results["llm"]["meta"][0]
Image(SEMANTIC_RAG_PIPELINE_IMG)

In [ ]:
Markdown(semantic_reply)